In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
import cmaps
import cartopy.feature as cfeature
from datetime import datetime,timedelta
from matplotlib.offsetbox import AnchoredText#关于修改text位置的
from tqdm import tqdm
from skimage.feature import peak_local_max#寻找矩阵局部极值点
# 用于提取点集凹边界
import alphashape
#计算任意地球上多边形面积
from area import area
import os
import psutil
from sklearn.neighbors import BallTree
proj =ccrs.NorthPolarStereo(central_longitude=15)
import ast
import calendar
import json
import seaborn as sns
import concurrent.futures


In [3]:
def chose_nordic_track(track,id_name):
    track.time = pd.to_datetime(track.time)
    track['month'] = [pd.Timestamp(i).month for i in track.time.values] 
    track['is_inmonth'] = ~np.isin(track.month,[11,12,1,2,3,4])
    month_bool = track.groupby(id_name)['is_inmonth'].sum()==0
    track_info = track.groupby(id_name)[['lon','lat','time']].agg(['max','min'])
    chosed_bool = (track_info.loc[:,'lat']['max']<85)&(track_info.loc[:,'lat']['min']>50)&\
        (track_info.loc[:,'lon']['max']<80)&(track_info.loc[:,'lon']['min']>-45)&(track_info.loc[:,'time']['min']>pd.Timestamp(2000,1,1,0))
    chosed_path_indexs = track_info.loc[chosed_bool.values&month_bool.values].index
    chosed_track = track.loc[np.isin(track.loc[:,id_name],chosed_path_indexs)].drop(columns=['is_inmonth','month'])
    return chosed_track

读取轨迹集信息

In [4]:
stoll_tracks = pd.read_csv(r"D:\Desktop\POLAR LOWS_WORK\A global climatology of polar lows_Stoll_2022\Climatology_Northern_Hemisphere.csv",usecols=['ID','step','lon','lat','time','rel_vort_850_smth','slp','vortex_diameter'])
stoll_tracks = chose_nordic_track(stoll_tracks,'ID')

In [5]:
stoll_Rojo_track = pd.read_csv(r"D:\Desktop\POLAR LOWS_WORK\A global climatology of polar lows_Stoll_2022\Rojo.csv",usecols=['ID','step','lon','lat','time','slp','vortex_diameter','PLlist_match'])
stoll_Rojo_track['time'] = pd.to_datetime(stoll_Rojo_track['time'])
stoll_Rojo_track

,ID,step,lon,lat,time,PLlist_match,vortex_diameter,slp
0,119991209500,0,13.25,75.00,1999-12-18 22:00:00,1,112.737435,993.60059
1,119991209500,1,13.50,74.75,1999-12-18 23:00:00,1,128.534147,993.23999
2,119991209500,2,13.75,74.50,1999-12-19 00:00:00,1,142.545977,993.37683
3,119991209500,3,14.00,74.25,1999-12-19 01:00:00,1,155.689657,992.83374
4,119991209500,4,14.50,74.00,1999-12-19 02:00:00,1,159.287305,992.98059
...,...,...,...,...,...,...,...,...
6522,120190500410,25,0.25,62.00,2019-05-03 02:00:00,263,166.249204,1006.57623
6523,120190500410,26,0.75,61.50,2019-05-03 03:00:00,263,162.216914,1005.75500
6524,120190500410,27,1.25,61.25,2019-05-03 04:00:00,263,165.148544,1005.26874
6525,120190500410,28,1.50,60.75,2019-05-03 05:00:00,263,158.517045,1005.08496


In [6]:
stoll_Noer_track = pd.read_csv(r"D:\Desktop\POLAR LOWS_WORK\A global climatology of polar lows_Stoll_2022\Noer.csv",usecols=['ID','step','lon','lat','time','slp','vortex_diameter','PLlist_match'])
stoll_Noer_track['time'] = pd.to_datetime(stoll_Noer_track['time'])
stoll_Noer_track

,ID,step,lon,lat,time,PLlist_match,vortex_diameter,slp
0,120020106230,0,2.25,72.75,2002-01-11 08:00:00,1,100.182190,985.69684
1,120020106230,1,3.25,73.00,2002-01-11 09:00:00,1,108.375051,985.30750
2,120020106230,2,4.25,73.25,2002-01-11 10:00:00,1,127.927314,985.17560
3,120020106230,3,6.00,73.25,2002-01-11 11:00:00,1,142.198685,985.02686
4,120020106230,4,6.75,73.50,2002-01-11 12:00:00,1,151.552949,984.39313
...,...,...,...,...,...,...,...,...
2695,120100310280,40,-11.50,63.50,2010-03-22 04:00:00,185,196.178078,999.82434
2696,120100310280,41,-11.75,63.75,2010-03-22 05:00:00,185,187.010241,999.70874
2697,120100310280,42,-12.25,64.00,2010-03-22 06:00:00,185,182.028731,999.86249
2698,120100310280,43,-13.50,64.00,2010-03-22 07:00:00,185,169.345646,999.68121


In [7]:
ROJO_tracks = pd.read_excel(r"D:\Desktop\POLAR LOWS_WORK\TRACKS_Rojo\Rojo-etal_1999_2019.xlsx",\
                       usecols=['ID','Time','Latitude','Longitude','Morphology','Stage','MSLP','diameter','Max_10m_10minute_wind']).rename({'Time':'time','Longitude':'lon','Latitude':'lat','Max_10m_10minute_wind':'wind10m_10min'},axis=1)
ROJO_tracks = chose_nordic_track(ROJO_tracks,'ID')
ROJO_tracks

,ID,time,lat,lon,diameter,Morphology,Stage,MSLP,wind10m_10min
7,2,2000-01-21 18:05:00,76.5,31.0,90.0,U,M,990,42
8,2,2000-01-22 02:49:00,72.5,29.0,170.0,S,M,990,42
9,2,2000-01-22 04:30:00,72.0,30.0,160.0,S,M,990,42
10,2,2000-01-22 06:11:00,72.0,29.5,220.0,H - C,M,990,42
11,2,2000-01-22 07:53:00,71.3,30.0,230.0,H - C,M,990,42
...,...,...,...,...,...,...,...,...,...
3827,262,2019-03-31 15:21:00,74.5,13.5,250.0,S,M,991,47
3828,262,2019-03-31 18:15:00,73.8,17.0,280.0,S,M,991,47
3829,262,2019-03-31 19:21:00,73.6,18.0,270.0,S,M,991,47
3830,262,2019-03-31 19:54:00,73.5,19.5,260.0,S,M,991,47


In [8]:
Noer_tracks = pd.DataFrame(np.loadtxt(r"D:\Desktop\PLs_identify\other_tracks\Noer.txt"),columns=['id','year','month','day','hour','min','lat','lon','Q','R(km)'])
dates = []
for idx,x in Noer_tracks.iterrows():
    dates.append(pd.Timestamp(year=int(x.year),month=int(x.month),day=int(x.day),hour=int(x.hour),minute=int(x['min'])))
Noer_tracks['time'] =dates
Noer_tracks = Noer_tracks.drop(columns=['year','month','day','hour','min'])
Noer_tracks = chose_nordic_track(Noer_tracks,'id')
Noer_tracks 

,id,lat,lon,Q,R(km),time
0,1.0,74.54,28.01,0.0,250.0,2002-01-12 01:00:00
1,1.0,74.51,29.44,0.0,250.0,2002-01-12 02:00:00
2,1.0,74.43,31.08,0.0,250.0,2002-01-12 03:00:00
3,1.0,74.28,32.99,0.0,250.0,2002-01-12 04:00:00
4,1.0,74.21,35.06,0.0,250.0,2002-01-12 05:00:00
...,...,...,...,...,...,...
1718,140.0,70.45,37.41,0.0,160.0,2011-03-31 12:00:00
1719,140.0,70.33,38.35,0.0,160.0,2011-03-31 13:00:00
1720,140.0,70.24,38.98,0.0,180.0,2011-03-31 14:00:00
1721,140.0,70.16,39.59,0.0,180.0,2011-03-31 15:00:00


读取自己的轨迹

In [9]:
def geodistance(lon1,lat1,lon2,lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2]) # 经纬度转换成弧度
    dlon=lon2-lon1
    dlat=lat2-lat1
    a=np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    distance=2*np.arcsin(np.sqrt(a))*6371*1000 # 地球平均半径，6371km
    distance=np.round(distance,3)
    return distance/1000
def deal_distance(line):
    distance = [0]
    for i in range(len(line)-1):
        distance.append(np.round(geodistance(line[i][0],line[i][1],line[i+1][0],line[i+1][1]),3))
    return distance

In [10]:
ERA5_tracks = pd.concat([pd.read_excel(r"Y:\whole_track_infos\split_all_tracks1.xlsx"),pd.read_excel(r"Y:\whole_track_infos\split_all_tracks2.xlsx")]).set_index('Unnamed: 0')
ERA5_tracks['split_path_index'] = ERA5_tracks['path_index']
ERA5_tracks

,point_index,time,path_index,peak_value,lon,lat,split_path_index
Unnamed: 0,,,,,,,
0,2,2000-01-01 22:00:00,1,1.685234,-10.50,58.75,1
1,3,2000-01-01 23:00:00,1,1.484584,-9.25,59.25,1
2,4,2000-01-02 00:00:00,1,1.282034,-6.75,59.50,1
3,5,2000-01-02 01:00:00,1,1.380362,-5.75,60.50,1
4,6,2000-01-02 02:00:00,1,1.699921,-5.00,60.75,1
...,...,...,...,...,...,...,...
528481,1233277,2024-04-29 08:00:00,46733,1.421962,12.75,66.25,46733
528482,1233278,2024-04-29 09:00:00,46733,1.323345,13.75,67.50,46733
528483,1233279,2024-04-29 10:00:00,46733,1.597069,14.00,67.50,46733


In [11]:
#分割距离较长的气旋路径
threshold = 200#KM,当距离超过200KM时认为路径断开，这种情况常常因为涡旋区域的中心发生了改变，这样处理认为改变后的涡旋为新生涡旋，原涡旋为衰退涡旋
path_reindex = ERA5_tracks.path_index.max()+1
new_track_record = []
for path_index in tqdm(ERA5_tracks.path_index.unique()):
    pick_track = ERA5_tracks.loc[ERA5_tracks.path_index==path_index]
    #计算轨迹中点与点之间的距离
    dis_interval = np.array(deal_distance(pick_track.loc[:,['lon','lat']].values))
    #找到分割点
    track_seg = np.concatenate((np.array([0]),np.where(dis_interval>threshold)[0],np.array([len(pick_track)])), axis=0)
    #分割不同部分
    for i in range(len(track_seg)-1):
        #第一段按照原来的Path_index
        temp_track = pick_track[track_seg[i]:track_seg[i+1]].copy()
        if (i>0)&(temp_track.iloc[0].peak_value<2):
            temp_track.split_path_index= path_reindex
            path_reindex += 1
        new_track_record.append(temp_track)      
#去除强度分割得到的中间path_index
ERA5_tracks = pd.concat(new_track_record,axis=0)

100%|██████████| 46002/46002 [01:22<00:00, 559.59it/s]


In [38]:
len(chose_nordic_track(ERA5_tracks,'split_path_index').split_path_index.unique())

57688

In [12]:
path_reindex

58425

轨迹记录

In [13]:
id_count = ROJO_tracks.value_counts('ID')
chosed_index = id_count[id_count>3].index
ROJO_tracks = ROJO_tracks.loc[np.isin(ROJO_tracks.ID,chosed_index)].rename({'ID':'path_index'},axis=1)
ROJO_tracks['integral_time'] = ROJO_tracks['time'].apply(lambda x:x-timedelta(seconds=x.minute*60)+timedelta(seconds=60*60)*round(x.minute/60))#将分钟去掉
ROJO_tracks = ROJO_tracks.rename({'time':'detailed_time','integral_time':'time'},axis=1)

In [14]:
#对ROJO的tracks创建一个dataframe储存路径
ROJO_tracks_record = pd.DataFrame(columns=['path_index','start_time','end_time','point_num','lifetime'])
for idx,path_index in tqdm(enumerate(ROJO_tracks.path_index.unique())):
    pick_track = ROJO_tracks.loc[ROJO_tracks.path_index==path_index]
    lifetime = pick_track.iloc[-1].time-pick_track.iloc[0].time
    hours = lifetime.total_seconds()/3600
    points_num = len(pick_track)
    ROJO_tracks_record.loc[idx,:] = [path_index,pick_track.iloc[0].time,pick_track.iloc[-1].time,points_num,hours]
ROJO_tracks_record

370it [00:00, 1345.41it/s]


,path_index,start_time,end_time,point_num,lifetime
0,2,2000-01-21 18:00:00,2000-01-22 08:00:00,5,14.0
1,3,2000-01-31 04:00:00,2000-01-31 13:00:00,4,9.0
2,5,2000-03-24 03:00:00,2000-03-24 18:00:00,7,15.0
3,6,2001-01-01 06:00:00,2001-01-02 14:00:00,8,32.0
4,7,2001-02-04 05:00:00,2001-02-04 16:00:00,7,11.0
...,...,...,...,...,...
365,259.h,2019-03-07 03:00:00,2019-03-07 15:00:00,7,12.0
366,260,2019-03-24 20:00:00,2019-03-25 20:00:00,16,24.0
367,261,2019-03-29 04:00:00,2019-03-29 12:00:00,9,8.0
368,261.b,2019-03-29 04:00:00,2019-03-29 10:00:00,6,6.0


In [15]:
Noer_tracks = pd.read_excel(r"D:\Desktop\PLs_identify\other_tracks\Noer_Nordic.xlsx").drop(columns=['Unnamed: 0']).rename({'id':'path_index'},axis=1)
id_count = Noer_tracks.value_counts('path_index')
chosed_index = id_count[id_count>3].index
Noer_tracks = Noer_tracks.loc[np.isin(Noer_tracks.path_index,chosed_index)]
Noer_tracks['integral_time'] = Noer_tracks['time'].apply(lambda x:x-timedelta(seconds=x.minute*60)+timedelta(seconds=60*60)*round(x.minute/60))#将分钟去掉
Noer_tracks = Noer_tracks.rename({'time':'detailed_time','integral_time':'time'},axis=1)
Noer_tracks

,path_index,lat,lon,Q,R(km),detailed_time,time
0,1,74.54,28.01,0.0,250,2002-01-12 01:00:00,2002-01-12 01:00:00
1,1,74.51,29.44,0.0,250,2002-01-12 02:00:00,2002-01-12 02:00:00
2,1,74.43,31.08,0.0,250,2002-01-12 03:00:00,2002-01-12 03:00:00
3,1,74.28,32.99,0.0,250,2002-01-12 04:00:00,2002-01-12 04:00:00
4,1,74.21,35.06,0.0,250,2002-01-12 05:00:00,2002-01-12 05:00:00
...,...,...,...,...,...,...,...
1665,140,70.45,37.41,0.0,160,2011-03-31 12:00:00,2011-03-31 12:00:00
1666,140,70.33,38.35,0.0,160,2011-03-31 13:00:00,2011-03-31 13:00:00
1667,140,70.24,38.98,0.0,180,2011-03-31 14:00:00,2011-03-31 14:00:00
1668,140,70.16,39.59,0.0,180,2011-03-31 15:00:00,2011-03-31 15:00:00


In [16]:
#对Noer的tracks创建一个dataframe储存路径
Noer_tracks_record = pd.DataFrame(columns=['path_index','start_time','end_time','point_num','lifetime'])
for idx,path_index in tqdm(enumerate(Noer_tracks.path_index.unique())):
    pick_track = Noer_tracks.loc[Noer_tracks.path_index==path_index]
    lifetime = pick_track.iloc[-1].time-pick_track.iloc[0].time
    hours = lifetime.total_seconds()/3600
    points_num = len(pick_track)
    Noer_tracks_record.loc[idx,:] = [path_index,pick_track.iloc[0].time,pick_track.iloc[-1].time,points_num,hours]
Noer_tracks_record

114it [00:00, 1310.36it/s]


,path_index,start_time,end_time,point_num,lifetime
0,1,2002-01-12 01:00:00,2002-01-12 16:00:00,16,15.0
1,2,2002-01-19 01:00:00,2002-01-19 06:00:00,6,5.0
2,5,2002-01-26 03:00:00,2002-01-27 08:00:00,30,29.0
3,6,2002-02-19 09:00:00,2002-02-19 16:00:00,8,7.0
4,7,2002-02-22 00:00:00,2002-02-22 08:00:00,9,8.0
...,...,...,...,...,...
109,136,2011-03-20 10:00:00,2011-03-21 10:00:00,25,24.0
110,137,2011-03-22 13:00:00,2011-03-22 18:00:00,6,5.0
111,138,2011-03-22 03:00:00,2011-03-23 14:00:00,36,35.0
112,139,2011-03-24 06:00:00,2011-03-24 14:00:00,9,8.0


In [17]:
#对stoll的tracks创建一个dataframe储存路径
stoll_tracks = stoll_tracks.rename({'ID':'path_index'},axis=1)
stoll_tracks_record = pd.DataFrame(columns=['start_time','end_time','lifetime','path_index'])
for idx,path_index in tqdm(enumerate(stoll_tracks.path_index.unique())):
    pick_track = stoll_tracks.loc[stoll_tracks.path_index==path_index]
    stoll_tracks_record.loc[idx,:] = [pick_track.iloc[0].time,pick_track.iloc[-1].time,len(pick_track),path_index]
stoll_tracks_record = stoll_tracks_record.sort_values(by='path_index')
stoll_tracks_record

3179it [00:01, 1671.34it/s]


,start_time,end_time,lifetime,path_index
0,2000-01-01 15:00:00,2000-01-03 07:00:00,41,120000100450
1,2000-01-02 02:00:00,2000-01-02 19:00:00,18,120000100600
2,2000-01-04 06:00:00,2000-01-04 13:00:00,8,120000101630
3,2000-01-06 08:00:00,2000-01-06 17:00:00,10,120000103060
4,2000-01-06 23:00:00,2000-01-07 16:00:00,18,120000103420
...,...,...,...,...
3174,2020-12-16 18:00:00,2020-12-17 14:00:00,21,120201209360
3175,2020-12-18 14:00:00,2020-12-19 07:00:00,18,120201210180
3176,2020-12-19 08:00:00,2020-12-20 02:00:00,19,120201210740
3177,2020-12-21 11:00:00,2020-12-22 04:00:00,18,120201211790


In [18]:
#对ERA5的tracks创建一个dataframe储存路径
ERA5_tracks_record = pd.DataFrame(columns=['start_time','end_time','lifetime','path_index'])
for idx,split_path_index in tqdm(enumerate(ERA5_tracks.split_path_index.unique())):
    pick_track = ERA5_tracks.loc[ERA5_tracks.split_path_index==split_path_index]
    ERA5_tracks_record.loc[idx,:] = [pick_track.iloc[0].time,pick_track.iloc[-1].time,len(pick_track),split_path_index]
ERA5_tracks_record = ERA5_tracks_record.sort_values(by='path_index')
ERA5_tracks_record

57693it [03:11, 301.48it/s]


,start_time,end_time,lifetime,path_index
0,2000-01-01 22:00:00,2000-01-03 05:00:00,32,1
1,2000-01-01 13:00:00,2000-01-03 07:00:00,43,2
3,2000-01-03 00:00:00,2000-01-04 15:00:00,40,3
4,2000-01-04 14:00:00,2000-01-07 06:00:00,65,4
5,2000-01-05 18:00:00,2000-01-08 00:00:00,55,5
...,...,...,...,...
57673,2024-04-11 05:00:00,2024-04-11 17:00:00,12,58420
57677,2024-04-15 12:00:00,2024-04-15 14:00:00,3,58421
57682,2024-04-20 01:00:00,2024-04-20 06:00:00,6,58422
57686,2024-04-21 10:00:00,2024-04-21 21:00:00,11,58423


匹配

In [19]:
def geodistance(lon1,lat1,lon2,lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2]) # 经纬度转换成弧度
    dlon=lon2-lon1
    dlat=lat2-lat1
    a=np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    distance=2*np.arcsin(np.sqrt(a))*6371*1000 # 地球平均半径，6371km
    distance=np.round(distance,3)
    return distance/1000

In [20]:
def between(array,small,large):
    return np.logical_and(array<=large,array>=small)
#找到与人工轨迹时间重合至少百分之六十的轨迹片段作为候选轨迹
def ERA5_track_filter(ERA5_tracks_record,target_track_demo,overlap_thresh=0.6):
    public_track = ERA5_tracks_record.copy()
    public_track['overlap_start_time'] = np.where(public_track['start_time']>target_track_demo.start_time,public_track['start_time'],target_track_demo.start_time)
    public_track['overlap_end_time'] = np.where(public_track['end_time']<target_track_demo.end_time,public_track['end_time'],target_track_demo.end_time)
    public_track['overlap_time']= [i.total_seconds()/3600 for i in (public_track['overlap_end_time']-public_track['overlap_start_time'])]
    public_track['matched_target_lifetime'] = target_track_demo.lifetime
    public_track['overlap_frac'] = public_track['overlap_time']/target_track_demo.lifetime
    public_track['target_start_time'] = target_track_demo.start_time
    public_track['target_end_time'] = target_track_demo.end_time
    return public_track.loc[public_track['overlap_frac']>overlap_thresh]

In [21]:
#传统方法:计算同一小时的距离，返回匹配的平均距离，以及小于150KM距离的小时与原路径时间占比
def simple_dis(target_points,ERA5_points):
    dis = []
    for index,point in target_points.iterrows():
        #找到对应时间的点
        matched_ERA5_point = ERA5_points.loc[ERA5_points.time==point.time]
        if len(matched_ERA5_point):
            dis.append(geodistance(point.lon,point.lat,matched_ERA5_point.iloc[0].lon,matched_ERA5_point.iloc[0].lat))
        else:
            dis.append(np.nan)
    dis = np.array(dis)
    #nan值参与判断时为False
    return np.nanmean(dis),np.sum(dis<150)/len(target_points)

In [22]:
def match_track(ERA5_tracks,ERA5_tracks_record,target_tracks,target_tracks_record):
    #id统一为:path_index
    matched_record = []
    for index,target_track_demo in tqdm(target_tracks_record.iterrows(),total=len(target_tracks_record),desc='匹配中'):
    #筛选可匹配的轨迹（重合度较高的）
        public_track = ERA5_track_filter(ERA5_tracks_record,target_track_demo)
        for j in range(len(public_track)):
            candidate_track = public_track.iloc[j].copy()
            candidate_track['matched_target_id'] = target_track_demo.path_index
            target_points = target_tracks.loc[target_tracks.path_index==target_track_demo.path_index]
            ERA5_points = ERA5_tracks.loc[ERA5_tracks.split_path_index==candidate_track.path_index]
            #普通匹配距离
            mean_dis,matched_ratio = simple_dis(target_points,ERA5_points)
            candidate_track['mean_dis'] = mean_dis
            candidate_track['matched_ratio'] = matched_ratio
            matched_record.append(pd.DataFrame(candidate_track).T)
    return matched_record

In [23]:
#找到最佳匹配关系，包括一个长对多个短的情况
def match_deal(matched_record,matched_ratio_thresh=0.5):
    matched = pd.concat(matched_record,axis=0)
    successful_matched = matched.loc[matched.matched_ratio>matched_ratio_thresh].reset_index(drop=True)
    #找到目标轨迹数据集中每个轨迹对应的最近ERA5轨迹
    best_matched_record  = successful_matched.loc[successful_matched.groupby('matched_target_id')['mean_dis'].idxmin()]
    #找到一个ERA5轨迹对应多个target轨迹的情况
    match_counts = best_matched_record.value_counts('path_index')
    multi_indexs = match_counts[match_counts>1].index
    multi_match = best_matched_record.loc[np.isin(best_matched_record.path_index,multi_indexs)]
    #判断一对多的情况,判断时间有没有重叠
    chosed_tracks = []
    for i in tqdm(range(len(multi_indexs))):
    #-----------------------------------------------------------------------------------------------
        multi_match_tracks = multi_match.loc[multi_match.path_index==multi_indexs[i]].copy()
        #倒置lifetime使得排序时先按照mean_dis从小到大排序，相同距离按照时间从大到小排序
        multi_match_tracks['matched_target_lifetime'] = -multi_match_tracks['matched_target_lifetime']
        multi_match_tracks = multi_match_tracks.sort_values(['mean_dis','matched_target_lifetime'])
        multi_match_tracks['matched_target_lifetime'] = -multi_match_tracks['matched_target_lifetime']
        #迭代选择非重叠时间的轨迹
        res_multi_match_tracks = multi_match_tracks.copy()
        while len(res_multi_match_tracks):
            #记录最近的非重叠时间以及对应匹配关系
            last_start_time,last_end_time = res_multi_match_tracks.iloc[0][['target_start_time','target_end_time']]
            chosed_tracks.append(res_multi_match_tracks.iloc[0].to_frame().T)
            #判断余下轨迹中时间非重叠的轨迹
            non_overlap_bool = np.logical_and(~between(res_multi_match_tracks.target_start_time,last_start_time,last_end_time),~between(res_multi_match_tracks.target_end_time,last_start_time,last_end_time))
            #更新
            res_multi_match_tracks = res_multi_match_tracks.loc[non_overlap_bool]
    return pd.concat([best_matched_record.loc[~np.isin(best_matched_record.path_index,multi_indexs)]]+chosed_tracks,axis=0)

In [ ]:
Rojo_matched = match_track(ERA5_tracks,ERA5_tracks_record,ROJO_tracks,ROJO_tracks_record)
Rojo_best_match = match_deal(Rojo_matched)


匹配中:  20%|██        | 75/370 [00:15<01:32,  3.19it/s]C:\Users\84648\AppData\Local\Temp\ipykernel_16892\1158298222.py:13: RuntimeWarning: Mean of empty slice
  return np.nanmean(dis),np.sum(dis<150)/len(target_points)
匹配中:  33%|███▎      | 122/370 [00:25<00:55,  4.50it/s]C:\Users\84648\AppData\Local\Temp\ipykernel_16892\1158298222.py:13: RuntimeWarning: Mean of empty slice
  return np.nanmean(dis),np.sum(dis<150)/len(target_points)
匹配中:  54%|█████▍    | 199/370 [00:41<00:34,  4.95it/s]C:\Users\84648\AppData\Local\Temp\ipykernel_16892\1158298222.py:13: RuntimeWarning: Mean of empty slice
  return np.nanmean(dis),np.sum(dis<150)/len(target_points)
匹配中:  73%|███████▎  | 271/370 [00:56<00:19,  5.04it/s]C:\Users\84648\AppData\Local\Temp\ipykernel_16892\1158298222.py:13: RuntimeWarning: Mean of empty slice
  return np.nanmean(dis),np.sum(dis<150)/len(target_points)
匹配中:  75%|███████▌  | 279/370 [00:58<00:26,  3.48it/s]C:\Users\84648\AppData\Local\Temp\ipykernel_16892\1158298222.py:13: Runtime

In [28]:
print("包含了%d个路径,匹配了路径有%d个,匹配率是%.2f%%"%(len(ROJO_tracks_record),len(Rojo_best_match),len(Rojo_best_match)/len(ROJO_tracks_record)*100))

包含了370个路径,匹配了路径有259个,匹配率是70.00%


In [ ]:
final_stoll_matched = match_track(ERA5_tracks,ERA5_tracks_record,stoll_tracks,stoll_tracks_record)
temp_stoll_best_match = match_deal(final_stoll_matched)
#选择匹配率大于0.8的匹配关系


匹配中:   3%|▎         | 81/3179 [00:19<10:49,  4.77it/s]C:\Users\84648\AppData\Local\Temp\ipykernel_16892\1158298222.py:13: RuntimeWarning: Mean of empty slice
  return np.nanmean(dis),np.sum(dis<150)/len(target_points)
匹配中:   7%|▋         | 217/3179 [00:50<09:27,  5.22it/s]C:\Users\84648\AppData\Local\Temp\ipykernel_16892\1158298222.py:13: RuntimeWarning: Mean of empty slice
  return np.nanmean(dis),np.sum(dis<150)/len(target_points)
匹配中:  13%|█▎        | 408/3179 [01:37<14:31,  3.18it/s]C:\Users\84648\AppData\Local\Temp\ipykernel_16892\1158298222.py:13: RuntimeWarning: Mean of empty slice
  return np.nanmean(dis),np.sum(dis<150)/len(target_points)
匹配中:  13%|█▎        | 409/3179 [01:37<12:27,  3.71it/s]C:\Users\84648\AppData\Local\Temp\ipykernel_16892\1158298222.py:13: RuntimeWarning: Mean of empty slice
  return np.nanmean(dis),np.sum(dis<150)/len(target_points)
匹配中:  29%|██▉       | 914/3179 [03:36<08:30,  4.44it/s]C:\Users\84648\AppData\Local\Temp\ipykernel_16892\1158298222.py:13: Ru

In [35]:
259/370

0.7

In [33]:
stoll_best_match = temp_stoll_best_match.loc[temp_stoll_best_match.matched_ratio>0.8]
print("包含了%d个路径,匹配了路径有%d个,匹配率是%.2f%%"%(len(stoll_tracks_record),len(stoll_best_match),len(stoll_best_match)/len(stoll_tracks_record)*100))

包含了3179个路径,匹配了路径有2881个,匹配率是90.63%


In [27]:
Noer_matched = match_track(ERA5_tracks,ERA5_tracks_record,Noer_tracks,Noer_tracks_record)
Noer_best_match = match_deal(Noer_matched)

匹配中:  72%|███████▏  | 82/114 [00:19<00:06,  4.84it/s]C:\Users\84648\AppData\Local\Temp\ipykernel_16892\1158298222.py:13: RuntimeWarning: Mean of empty slice
  return np.nanmean(dis),np.sum(dis<150)/len(target_points)
匹配中:  73%|███████▎  | 83/114 [00:19<00:06,  5.09it/s]C:\Users\84648\AppData\Local\Temp\ipykernel_16892\1158298222.py:13: RuntimeWarning: Mean of empty slice
  return np.nanmean(dis),np.sum(dis<150)/len(target_points)
匹配中:  75%|███████▍  | 85/114 [00:19<00:05,  5.36it/s]C:\Users\84648\AppData\Local\Temp\ipykernel_16892\1158298222.py:13: RuntimeWarning: Mean of empty slice
  return np.nanmean(dis),np.sum(dis<150)/len(target_points)
C:\Users\84648\AppData\Local\Temp\ipykernel_16892\1158298222.py:13: RuntimeWarning: Mean of empty slice
  return np.nanmean(dis),np.sum(dis<150)/len(target_points)
匹配中:  78%|███████▊  | 89/114 [00:20<00:07,  3.19it/s]C:\Users\84648\AppData\Local\Temp\ipykernel_16892\1158298222.py:13: RuntimeWarning: Mean of empty slice
  return np.nanmean(dis),np.

In [34]:
print("包含了%d个路径,匹配了路径有%d个,匹配率是%.2f%%"%(len(Noer_tracks_record),len(Noer_best_match),len(Noer_best_match)/len(Noer_tracks_record)*100))

包含了114个路径,匹配了路径有100个,匹配率是87.72%


In [23]:
stoll_best_match 

,start_time,end_time,lifetime,path_index,overlap_start_time,overlap_end_time,overlap_time,matched_target_lifetime,overlap_frac,target_start_time,target_end_time,matched_target_id,mean_dis,matched_ratio
2,2002-01-10 12:00:00,2002-01-11 18:00:00,31,266,2002-01-11 05:00:00,2002-01-11 16:00:00,11.0,12,0.916667,2002-01-11 05:00:00,2002-01-11 16:00:00,120020106170,7.091951,1.0
3,2002-01-11 07:00:00,2002-01-13 20:00:00,62,268,2002-01-11 08:00:00,2002-01-13 04:00:00,44.0,45,0.977778,2002-01-11 08:00:00,2002-01-13 04:00:00,120020106230,11.446876,1.0
5,2002-01-21 02:00:00,2002-01-22 05:00:00,28,273,2002-01-21 04:00:00,2002-01-21 23:00:00,19.0,20,0.95,2002-01-21 04:00:00,2002-01-21 23:00:00,120020110940,16.703977,1.0
6,2002-01-21 21:00:00,2002-01-22 16:00:00,20,274,2002-01-22 01:00:00,2002-01-22 14:00:00,13.0,14,0.928571,2002-01-22 01:00:00,2002-01-22 14:00:00,120020111560,7.292631,1.0
7,2002-01-23 03:00:00,2002-01-23 20:00:00,18,278,2002-01-23 04:00:00,2002-01-23 16:00:00,12.0,13,0.923077,2002-01-23 04:00:00,2002-01-23 16:00:00,120020112100,10.662477,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,2012-03-24 13:00:00,2012-03-28 21:00:00,105,56410,2012-03-24 13:00:00,2012-03-25 12:00:00,23.0,24,0.958333,2012-03-24 13:00:00,2012-03-25 12:00:00,120120311760,49.405898,0.916667
782,2013-11-22 13:00:00,2013-11-25 09:00:00,69,57281,2013-11-22 13:00:00,2013-11-22 23:00:00,10.0,13,0.769231,2013-11-22 11:00:00,2013-11-22 23:00:00,120131112800,13.768584,0.846154
783,2013-11-22 13:00:00,2013-11-25 09:00:00,69,57281,2013-11-23 02:00:00,2013-11-24 14:00:00,36.0,37,0.972973,2013-11-23 02:00:00,2013-11-24 14:00:00,120131113140,49.676298,0.864865
639,2011-03-21 02:00:00,2011-03-23 22:00:00,69,1467,2011-03-21 04:00:00,2011-03-22 20:00:00,40.0,41,0.97561,2011-03-21 04:00:00,2011-03-22 20:00:00,120110310820,15.661838,0.97561
